In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [86]:
inventory_data = pd.read_excel('./data/inventory_data_new.xlsx')

In [87]:
inventory_data.head()

,material,unitats_2022,vendes_2022,preu_venda_unitari_2022,unitats_2023,vendes_2023,preu_venda_unitari_2023,variacio_preu_venda_unitari_2022_2023,proporcio_variacio_preu_venda_unitari_2022_2023,data_darrera_entrada,dies_ultima_entrada,data_darrera_sortida,dies_ultima_sortida,diferencia_entrada_sortida,stock_final_2023,valor_total_stock_2023,cost_unitari_stock_2023
0,100500401.0,350.0,766.77,2.190771,300.0,931.00,3.103333,0.912562,0.416548,1970-01-01 01:00:45.208,83.0,1970-01-01 01:00:45.219,72.0,11.0,NaN,NaN,NaN
1,100500601.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970-01-01 01:00:44.551,740.0,1970-01-01 01:00:44.551,740.0,0.0,NaN,NaN,NaN
2,114230204.0,140000.0,8072.60,0.057661,70000.0,4489.80,0.064140,0.006479,0.112355,1970-01-01 01:00:45.258,33.0,1970-01-01 01:00:45.272,19.0,14.0,NaN,NaN,NaN
3,114230821.0,40000.0,1945.60,0.048640,NaN,NaN,NaN,NaN,NaN,1970-01-01 01:00:44.581,710.0,1970-01-01 01:00:44.588,703.0,7.0,NaN,NaN,NaN
4,114230822.0,1018500.0,54892.49,0.053895,885500.0,58951.87,0.066575,0.012679,0.235256,1970-01-01 01:00:45.351,-60.0,1970-01-01 01:00:45.336,-45.0,-15.0,66500.0,2292.96,0.034481


In [88]:
# Passar a data les columnes que ho requereixen
inventory_data['data_darrera_entrada'] = pd.to_datetime(inventory_data['data_darrera_entrada'], format='%Y-%m-%d')
inventory_data['data_darrera_sortida'] = pd.to_datetime(inventory_data['data_darrera_sortida'], format='%Y-%m-%d')

In [89]:
categorical_cols = ['material']
date_cols = ['data_darrera_entrada', 'data_darrera_sortida']
numerical_cols = list(set(inventory_data.columns) - set(categorical_cols) - set(date_cols))
for col in numerical_cols:
	inventory_data[col] = pd.to_numeric(inventory_data[col])

In [90]:
inventory_data.columns

Index(['material', 'unitats_2022', 'vendes_2022', 'preu_venda_unitari_2022',
       'unitats_2023', 'vendes_2023', 'preu_venda_unitari_2023',
       'variacio_preu_venda_unitari_2022_2023',
       'proporcio_variacio_preu_venda_unitari_2022_2023',
       'data_darrera_entrada', 'dies_ultima_entrada', 'data_darrera_sortida',
       'dies_ultima_sortida', 'diferencia_entrada_sortida', 'stock_final_2023',
       'valor_total_stock_2023', 'cost_unitari_stock_2023'],
      dtype='object')

In [91]:
# Eliminar filas si tiene missings

inventory_data = inventory_data.dropna()

In [92]:
def calcular_indice_deterioro(data):

    # Calcular las diferencias y ratios asegurando que no haya división por cero
    data['delta_unitats'] = (data['unitats_2023'] - data['unitats_2022']) / (data['unitats_2022'] + 1)
    data['delta_preu_venda_unitari'] = data['variacio_preu_venda_unitari_2022_2023'] / (data['preu_venda_unitari_2022'] + 0.01)
    data['rotacion_inventario'] = data['unitats_2023'] / (data['stock_final_2023'] + 1)
    data['inactividad'] = data['dies_ultima_sortida'] + data['dies_ultima_entrada']

    # Índice de deterioro
    data['indice_deterioro'] = (
        -0.3 * data['delta_unitats'] +
        -0.2 * data['delta_preu_venda_unitari'] +
         0.2 * np.log1p(data['inactividad']) +
        -0.3 * data['rotacion_inventario']
    )
    
    # Asegurarse de que no haya NaN en el resultado final del índice de deterioro
    data['indice_deterioro'].fillna(0, inplace=True)

    return data

inventory_data = calcular_indice_deterioro(inventory_data)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [93]:
# Guardar el dataset con el índice de deterioro
inventory_data.to_csv('./data/inventory_data_with_deterioration_index.csv', index=False)